In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 65.1 MB/s eta 0:00:00


In [ ]:
pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00


In [20]:
import openai
import streamlit as st
import pandas as pd
import numpy as np
import os
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory
from typing import Dict

# Step 1: Initialize OpenAI API
os.environ["OPENAI_API_KEY"] = OpenAI() # Set your OpenAI API key here

# Step 2: Define the user questionnaire
def user_questionnaire() -> Dict[str, str]:
    st.title("Investment Planning Assistant")
    st.write("Answer the following questions to help us understand your financial profile.")

    age = st.number_input("What is your age?", min_value=18, max_value=100, step=1)
    annual_income = st.number_input("What is your annual income (in USD)?", min_value=0, step=1000)
    risk_tolerance = st.selectbox("What is your risk tolerance?", ["Low", "Medium", "High"])
    investment_goal = st.text_area("What is your primary investment goal? (e.g., retirement, education, buying a house)")
    investment_horizon = st.number_input("What is your investment horizon (in years)?", min_value=1, max_value=50, step=1)

    if st.button("Submit Questionnaire"):
        return {
            "age": age,
            "annual_income": annual_income,
            "risk_tolerance": risk_tolerance,
            "investment_goal": investment_goal,
            "investment_horizon": investment_horizon,
        }

    return {}

# Step 3: Generate personalized investment strategy
def generate_investment_strategy(profile: Dict[str, str], retriever):
    prompt = PromptTemplate(
        template=(
            "Given the following customer profile, provide a personalized investment strategy: \n"
            "Age: {age} \n"
            "Annual Income: {annual_income} \n"
            "Risk Tolerance: {risk_tolerance} \n"
            "Investment Goal: {investment_goal} \n"
            "Investment Horizon: {investment_horizon} \n"
            "Be specific and provide actionable advice."
        ),
        input_variables=["age", "annual_income", "risk_tolerance", "investment_goal", "investment_horizon"],
    )

    query = prompt.format(**profile)

    # Use LangChain RAG to retrieve context and generate a response
    qa_chain = RetrievalQA.from_chain_type(llm=openai.ChatCompletion(), retriever=retriever)
    response = qa_chain.run(query)

    return response

# Step 4: Integrate financial data for real-time updates
def fetch_financial_data():
    data = {
        "Asset": ["S&P 500 ETF", "Gold", "Bonds", "Real Estate REIT", "Tech Growth Fund"],
        "Price": [4500, 1900, 100, 250, 75],
        "1Y Return (%)": [12.5, -5.3, 3.0, 8.5, 25.7],
    }
    return pd.DataFrame(data)

# Step 5: Create an interactive dashboard
def dashboard(profile, strategy):
    st.header("Your Investment Strategy")
    st.write(strategy)

    st.header("Current Market Data")
    market_data = fetch_financial_data()
    st.table(market_data)

    st.header("Portfolio Allocation Suggestion")
    allocation = {
        "Stocks": 50 if profile["risk_tolerance"] == "High" else 30,
        "Bonds": 20 if profile["risk_tolerance"] == "High" else 50,
        "Gold": 15,
        "Real Estate": 15,
    }
    allocation_df = pd.DataFrame(
        {
            "Asset Class": allocation.keys(),
            "Percentage": allocation.values(),
        }
    )
    st.bar_chart(allocation_df.set_index("Asset Class"))

# Step 6: Create a chat interface with memory
def chat_interface():
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    st.title("Chat with Your Financial Advisor")
    user_message = st.text_input("Ask your financial advisor a question...")

    if 'messages' not in st.session_state:
        st.session_state.messages = []

    # Display previous messages in the chat
    for msg in st.session_state.messages:
        st.write(f"**{msg['role']}**: {msg['content']}")

    # Add user input to messages history
    if user_message:
        st.session_state.messages.append({"role": "user", "content": user_message})

        # Generate response with memory
        prompt = f"User: {user_message}\nAI:"
        chat_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=[
                {"role": "system", "content": "You are a financial advisor."},
                {"role": "user", "content": prompt},
            ]
        )
        ai_message = chat_response['choices'][0]['message']['content']
        st.session_state.messages.append({"role": "assistant", "content": ai_message})

        # Display AI response
        st.write(f"**AI**: {ai_message}")

# Step 7: Main App Logic
def main():
    # Create mock documents
    documents = [
        Document(page_content="The S&P 500 ETF is a broad-market equity index fund tracking the performance of the 500 largest publicly traded companies in the U.S."),
        Document(page_content="Gold is often seen as a safe haven asset, providing a hedge against inflation and market volatility."),
        Document(page_content="Bonds are debt instruments used by governments and corporations to raise capital."),
        Document(page_content="Real Estate Investment Trusts (REITs) allow you to invest in real estate without owning physical property."),
        Document(page_content="Tech Growth Funds are high-risk, high-reward funds that invest in emerging technology companies.")
    ]

    # Initialize embeddings and retriever
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)
    retriever = vectorstore.as_retriever()

    # Get user profile from questionnaire
    user_profile = user_questionnaire()

    if user_profile:
        st.write("Generating your personalized investment strategy...")

        # Generate investment strategy
        strategy = generate_investment_strategy(user_profile, retriever)

        # Show the dashboard
        dashboard(user_profile, strategy)

    # Display chat interface
    chat_interface()

if __name__ == "__main__":
    # Run the Streamlit app directly
    main()


2025-01-08 18:24:05.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.477 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.479 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 18:24:05.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar